In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display,clear_output

import requests
import time
from requests_html import HTMLSession
from PIL import Image
from io import BytesIO

In [ ]:
#creating a new driver
s=Service('E:\datascienceBootcamp/chromedriver')
driver=webdriver.Chrome(service=s)

In [ ]:
#fetching airbnb beach link
driver.get("https://www.airbnb.com/?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&search_mode=flex_destinations_search&flexible_trip_lengths%5B%5D=o

In [ ]:
def scroll_window():
#     total_height = int(driver.execute_script("return document.body.scrollHeight"))
    total_height = 33000

    for i in range(1, total_height, 700):
        driver.execute_script("window.scrollTo(0, {});".format(i))
        time.sleep(3)

In [ ]:
#collect all the destination links and store them in a file
scroll_window()
time.sleep(3)
destinations_link=driver.find_elements(by=By.CLASS_NAME,value="ln2bl2p")
print(len(destinations_link))
with open("destinations.txt","w") as f:
    for i in destinations_link:
        f.write(i.get_attribute('href'))
        f.write("\n")

functions for individual destination link

In [ ]:
#function for destination title
def titleHeading(driver):
    try:
        title=driver.find_element(by=By.CLASS_NAME,value="_fecoyn4").text
        print("title",title)
        return title
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
#function for destination rating
def placeRating(driver):
    try:
        rating=driver.find_element(by=By.CLASS_NAME,value="_17p6nbba").text
        print("rating",rating)
        return rating
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
#function for destination total reviews
def placeTotalReviw(driver):
    try:
        reviewNumber=driver.find_element(by=By.CLASS_NAME,value="_11eqlma4").text
        print("total_reviews",reviewNumber)
        return reviewNumber
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
#function for destination location
def placeLocation(driver):
    try:
        location=driver.find_element(by=By.CLASS_NAME,value="_9xiloll").text
        print("Location",location)
        return location
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
#function for destination price
def placeRent(driver):
    try:
        time.sleep(3)
        rent=driver.find_element(by=By.XPATH,value="//*/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]").text
        print("Place_rent",rent)
        return rent
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
#function for destination description
def placeDescription(driver):
    try:
        if(show_more_click(driver,"desc")==True):
            time.sleep(3)
            description =WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[11]/section/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div/div/section"))).text
            driver.back()
        else:
            description=driver.find_element(by=By.XPATH,value="//*/div/div[1]/div[3]/div/div[1]/div/div[4]/div/div[2]/div/span/span").text        
#         driver.refresh()
        print("Place_description",description)
        return description
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
#function for destination offers
def placeOffers(driver):
    try:
        show_more_click(driver,"feature")
        time.sleep(5)
        
        offers =WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[11]/section/div/div/div[2]/div/div[3]/div/div/div/div/section"))).text
        driver.back()
#         driver.refresh()
        print("Place_offers",offers)
        return offers
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
# function for show more click
def show_more_click(driver,specify):
#     print(type(specify))
#     print(specify=="desc")
    try:
        if specify=="img":
            driver.find_element(by=By.CLASS_NAME,value="_uhxsfg").click()
        elif specify=="desc":
            driver.find_element(by=By.XPATH,value="//*/div/div[1]/div[3]/div/div[1]/div/div[5]/div/div/div[2]/button/span/span[1]").click()
        elif specify=="feature":
            driver.find_element(by=By.CLASS_NAME,value="b65jmrv").click()
        return True
    except:
        print("no show more button")
        return False

In [ ]:
#function for destination reviews
def placeReviews(driver):
    try:
        reviews=driver.find_elements(by=By.CLASS_NAME,value="r1rl3yjt")
#         print(len(reviewss))
        review_text=[" ".join(i.text.split("\n")) for i in reviews]
        Reviews=" ".join(review_text)
        print("reviews",Reviews)
        return Reviews
    except:
        print("destination not found , may be link is broken")
        return None

In [ ]:
#open driver
def openDriver():
    op=webdriver.ChromeOptions()
    op.add_argument("headless")
    s=Service('E:\datascienceBootcamp/chromedriver')
    driver=webdriver.Chrome(service=s,options=op)
    return driver

In [ ]:
#function for homepage img scrapping
def pics_scrap(driver,category):
    time.sleep(3)
    images=driver.find_element(by=By.CLASS_NAME,value="_88xxct")
    time.sleep(2)
    img_names=[]
    imageUrls=[]
    title=attachTitle(driver)
    counter=0
    for img in images.find_elements(by=By.TAG_NAME,value='img')[:5]:
        img_urls = img.get_attribute('src')
        print(img_urls)
        imageUrls.append(img_urls)
        #     removing point (.) as this point causing a problem in saving the picture
        removingPoint=("/".join(img_urls.split("/")[-1:])).split('.')[:-1]
        img_urls_firstname = " ".join(removingPoint[0].split(" "))
        img_urls_rename= f"{title}/"+img_urls_firstname
        #             print(img_urls_rename)
        img_name = "@".join(img_urls_rename.split('/'))
        img_names.append(img_name)
#         if img_urls.mode != 'RGB':
#             img_urls = img_urls.convert('RGB')
        r = requests.get(img_urls)
        i = Image.open(BytesIO(r.content))
        if i.mode != 'RGB':
            i = i.convert('RGB')
        i.save(f"img/{category}/{img_name}.jpg")
        counter+=1
        print(img_name)
#     print(img_names)
    return img_names

In [ ]:
#getting the titles for attaching with the picture name
def attachTitle(driver):
    try:
        Title=driver.find_element(by=By.XPATH,value="//*/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[1]/div[2]/h2").text
        Title="_".join(Title.split(" "))
        return Title
    except:
        print("nothing found")
        return None

In [ ]:
#function for single product scrapping
def product_scrap(driver,url,category):
    driver.get(url)
    time.sleep(5)
    #single page data scrapping
    title=titleHeading(driver)
    #place rating
    rating=placeRating(driver)
    #place total review
    total_review=placeTotalReviw(driver)
    #place rent
    rent=placeRent(driver)
#     place location
    location=placeLocation(driver)
    #place descriptio
    description=placeDescription(driver)
    #place extra feature
    time.sleep(2)
    extra_offers=placeOffers(driver)
    #all reviews of the place
    time.sleep(2)
    reviews=placeReviews(driver)
    time.sleep(2)
    #all images
    imageName=pics_scrap(driver,category)
    time.sleep(2)
    return title,rating,total_review,rent,location,description,extra_offers,reviews,imageName

In [ ]:
#all link scrapping
category="destination_images"
with open("destinations.txt","r") as f:
    single_url=f.readlines()
    titles,ratings,total_reviews,rents,locations,descriptions,Extra_offers,Reviews,imageNames=[],[],[],[],[],[],[],[],[]
    airbnb_scrap_partwise(0,98,titles,ratings,total_reviews,rents,locations,descriptions,Extra_offers,Reviews,imageNames)
    time.sleep(30)
    airbnb_scrap_partwise(99,180,titles,ratings,total_reviews,rents,locations,descriptions,Extra_offers,Reviews,imageNames)
    time.sleep(30)
    airbnb_scrap_partwise(181,224,titles,ratings,total_reviews,rents,locations,descriptions,Extra_offers,Reviews,imageNames)

In [ ]:
#function for scrapping a limited number of links
def airbnb_scrap_partwise(num1,num2,titles,ratings,total_reviews,rents,locations,descriptions,Extra_offers,Reviews,imageNames):
    cnt=1
    for s in single_url[num1:num2]:
        print(cnt,s)
        driver=openDriver()
        title,rating,total_review,rent,location,description,extra_offers,reviews,imageName=product_scrap(driver,s,category)
        if title:
            titles.append(title)
            ratings.append(rating)
            total_reviews.append(total_review)
            rents.append(rent)
            locations.append(location)
            descriptions.append(description)
            Extra_offers.append(extra_offers)
            Reviews.append(reviews)
            imageNames.append(imageName)
            
            cnt+=1
            print("=============================================")
            clear_output(wait=True)

In [ ]:
#saving the scrapping infos
with open("Airbnb_Scrab.txt","w",encoding="utf-8") as f:
    f.write("No ,Title ,Ratings ,Total_reviews ,Rents ,Location ,Description ,Extra_offers ,Reviews ,Images")
    f.write("\n\n")
    for i,t,r,tr,ren,l,d,e,rev,im in zip(range(1,len(titles)+1),titles,ratings,total_reviews,rents,locations,descriptions,Extra_offers,Reviews,imageNames):
        f.write(f"{i} ,{t} ,{r} ,{tr} ,{ren} ,{l} ,{d} ,{e} ,{rev} ,{im} ")
        f.write("\n\n")